In [1]:
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
from typing import List, Dict, Optional
import json

class Cargo:
    def __init__(self, id: str, name: str, weight: float, volume: float,
                 fragile: bool = False, hazardous: bool = False):
        self.id = id
        self.name = name
        self.weight = weight
        self.volume = volume
        self.fragile = fragile
        self.hazardous = hazardous

class Warehouse:
    def __init__(self, id: str, name: str, location: str, capacity: float):
        self.id = id
        self.name = name
        self.location = location
        self.capacity = capacity
        self.current_load = 0.0

    def can_accept_cargo(self, cargo: Cargo):
        return self.current_load + cargo.weight <= self.capacity

    def load_cargo(self, cargo: Cargo):
        if self.can_accept_cargo(cargo):
            self.current_load += cargo.weight
            return True
        return False

    def unload_cargo(self, cargo: Cargo):
        self.current_load = max(0, self.current_load - cargo.weight)

class Transport(ABC):
    def __init__(self, id: str, name: str, capacity: float, max_weight: float,
                 speed: float, cost_per_km: float):
        self.id = id
        self.name = name
        self.capacity = capacity
        self.max_weight = max_weight
        self.speed = speed
        self.cost_per_km = cost_per_km

    @abstractmethod
    def calculate_cost(self, distance: float, cargo: Cargo) -> float:
        pass

    @abstractmethod
    def calculate_time(self, distance: float) -> timedelta:
        pass

    def can_carry(self, cargo: Cargo) -> bool:
        return cargo.weight <= self.max_weight and cargo.volume <= self.capacity

class Truck(Transport):
    def calculate_cost(self, distance: float, cargo: Cargo) -> float:
        base_cost = distance * self.cost_per_km
        if cargo.fragile:
            base_cost *= 1.3
        if cargo.hazardous:
            base_cost *= 1.5
        return base_cost

    def calculate_time(self, distance: float) -> timedelta:
        hours = distance / self.speed
        return timedelta(hours=hours)

class Ship(Transport):
    def calculate_cost(self, distance: float, cargo: Cargo) -> float:
        base_cost = distance * self.cost_per_km
        volume_discount = min(0.5, cargo.volume / 1000)
        base_cost *= (1 - volume_discount)
        return base_cost

    def calculate_time(self, distance: float) -> timedelta:
        hours = distance / self.speed
        return timedelta(hours=hours)

class Train(Transport):
    def calculate_cost(self, distance: float, cargo: Cargo) -> float:
        base_cost = distance * self.cost_per_km
        weight_discount = min(0.3, cargo.weight / 10000)
        base_cost *= (1 - weight_discount)
        return base_cost

    def calculate_time(self, distance: float) -> timedelta:
        hours = distance / self.speed
        return timedelta(hours=hours)

class Airplane(Transport):
    def __init__(self, id: str, name: str, capacity: float, max_weight: float,
                 speed: float, cost_per_km: float, range_limit: float):
        super().__init__(id, name, capacity, max_weight, speed, cost_per_km)
        self.range_limit = range_limit

    def calculate_cost(self, distance: float, cargo: Cargo) -> float:
        if distance > self.range_limit:
            return float('inf')

        base_cost = distance * self.cost_per_km
        base_cost *= 2.0
        if cargo.fragile or cargo.hazardous:
            base_cost *= 1.2
        return base_cost

    def calculate_time(self, distance: float) -> timedelta:
        if distance > self.range_limit:
            return timedelta(days=365)

        hours = distance / self.speed
        return timedelta(hours=hours)

class Route:
    def __init__(self, from_warehouse: Warehouse, to_warehouse: Warehouse, distance: float):
        self.from_warehouse = from_warehouse
        self.to_warehouse = to_warehouse
        self.distance = distance

class Delivery:
    def __init__(self, id: str, cargo: Cargo, route: Route, transport: Transport, start_time: datetime):
        self.id = id
        self.cargo = cargo
        self.route = route
        self.transport = transport
        self.start_time = start_time
        self.cost = transport.calculate_cost(route.distance, cargo)
        self.delivery_time = transport.calculate_time(route.distance)
        self.end_time = start_time + self.delivery_time
        self.status = "в пути"

class MultiDelivery:
    def __init__(self, id: str, cargo: Cargo, routes: List[Route], 
                 transports: List[Transport], start_time: datetime):
        self.id = id
        self.cargo = cargo
        self.routes = routes
        self.transports = transports
        self.start_time = start_time
        self.total_cost = 0
        self.total_time = timedelta()
        
        for i, (route, transport) in enumerate(zip(routes, transports)):
            segment_cost = transport.calculate_cost(route.distance, cargo)
            segment_time = transport.calculate_time(route.distance)
            self.total_cost += segment_cost
            self.total_time += segment_time
            
            if i < len(routes) - 1:
                self.total_time += timedelta(hours=2)
        
        self.end_time = start_time + self.total_time
        self.status = "в пути"

class TransportationManager:
    def __init__(self):
        self.warehouses = {}
        self.transports = {}
        self.deliveries = {}
        self.routes = []
        self.cargos = {}

    def add_warehouse(self, warehouse: Warehouse):
        self.warehouses[warehouse.id] = warehouse

    def add_transport(self, transport: Transport):
        self.transports[transport.id] = transport

    def add_cargo(self, cargo: Cargo):
        self.cargos[cargo.id] = cargo

    def add_route(self, route: Route):
        self.routes.append(route)

    def find_route(self, from_location: str, to_location: str):
        for route in self.routes:
            if (route.from_warehouse.location == from_location and
                route.to_warehouse.location == to_location):
                return route
        return None

    def find_routes_chain(self, from_location: str, to_location: str, max_hops: int = 3):
        def dfs(current_location: str, target_location: str, visited: set, 
                path: List[Route], all_paths: List[List[Route]], depth: int):
            if depth > max_hops:
                return
            
            if current_location == target_location and path:
                all_paths.append(path.copy())
                return
            
            for route in self.routes:
                if (route.from_warehouse.location == current_location and 
                    route.to_warehouse.location not in visited):
                    visited.add(route.to_warehouse.location)
                    path.append(route)
                    dfs(route.to_warehouse.location, target_location, 
                        visited, path, all_paths, depth + 1)
                    path.pop()
                    visited.remove(route.to_warehouse.location)
        
        all_paths = []
        dfs(from_location, to_location, set([from_location]), [], all_paths, 0)
        return all_paths

    def create_delivery(self, delivery_id: str, cargo_id: str,
                       from_location: str, to_location: str,
                       transport_id: str, start_time: datetime):
        cargo = self.cargos.get(cargo_id)
        if not cargo:
            return None
            
        route = self.find_route(from_location, to_location)
        if not route:
            return None

        transport = self.transports.get(transport_id)
        if not transport:
            return None

        if not transport.can_carry(cargo):
            return None

        if not route.from_warehouse.can_accept_cargo(cargo):
            return None

        delivery = Delivery(delivery_id, cargo, route, transport, start_time)
        self.deliveries[delivery_id] = delivery
        route.from_warehouse.load_cargo(cargo)
        
        return delivery

    def create_multi_delivery(self, delivery_id: str, cargo_id: str,
                            route_chain: List[Route], transports: List[str],
                            start_time: datetime):
        cargo = self.cargos.get(cargo_id)
        if not cargo:
            return None
            
        if len(route_chain) != len(transports):
            return None

        transport_objects = []
        for transport_id in transports:
            transport = self.transports.get(transport_id)
            if not transport or not transport.can_carry(cargo):
                return None
            transport_objects.append(transport)

        multi_delivery = MultiDelivery(delivery_id, cargo, route_chain, 
                                     transport_objects, start_time)
        self.deliveries[delivery_id] = multi_delivery
        return multi_delivery

    def get_delivery_options(self, cargo_id: str, from_location: str, to_location: str):
        cargo = self.cargos.get(cargo_id)
        if not cargo:
            return []

        route = self.find_route(from_location, to_location)
        if not route:
            return []

        options = []
        for transport in self.transports.values():
            if transport.can_carry(cargo):
                cost = transport.calculate_cost(route.distance, cargo)
                time = transport.calculate_time(route.distance)
                if cost != float('inf') and time.days < 365:
                    options.append({
                        'transport': transport.name,
                        'transport_id': transport.id,
                        'cost': cost,
                        'time': time
                    })
        return sorted(options, key=lambda x: x['cost'])

    def get_multi_delivery_options(self, cargo_id: str, from_location: str, to_location: str):
        cargo = self.cargos.get(cargo_id)
        if not cargo:
            return []

        route_chains = self.find_routes_chain(from_location, to_location)
        options = []

        for chain in route_chains:
            if len(chain) > 1:
                for transport1 in self.transports.values():
                    if not transport1.can_carry(cargo):
                        continue
                    
                    for transport2 in self.transports.values():
                        if not transport2.can_carry(cargo):
                            continue
                        
                        cost1 = transport1.calculate_cost(chain[0].distance, cargo)
                        time1 = transport1.calculate_time(chain[0].distance)
                        cost2 = transport2.calculate_cost(chain[1].distance, cargo)
                        time2 = transport2.calculate_time(chain[1].distance)
                        
                        if (cost1 != float('inf') and cost2 != float('inf') and 
                            time1.days < 365 and time2.days < 365):
                            
                            total_cost = cost1 + cost2
                            total_time = time1 + time2 + timedelta(hours=2)
                            
                            options.append({
                                'route_chain': [r.from_warehouse.location + " → " + r.to_warehouse.location for r in chain],
                                'transports': [transport1.name, transport2.name],
                                'transport_ids': [transport1.id, transport2.id],
                                'cost': total_cost,
                                'time': total_time
                            })
        
        return sorted(options, key=lambda x: x['cost'])

    def save_to_json(self, filename: str):
        data = {
            'warehouses': {id: {
                'id': wh.id, 'name': wh.name, 'location': wh.location, 
                'capacity': wh.capacity, 'current_load': wh.current_load
            } for id, wh in self.warehouses.items()},
            'transports': {id: trans.to_dict() if hasattr(trans, 'to_dict') else {
                'type': trans.__class__.__name__, 'id': trans.id, 'name': trans.name,
                'capacity': trans.capacity, 'max_weight': trans.max_weight,
                'speed': trans.speed, 'cost_per_km': trans.cost_per_km,
                'range_limit': getattr(trans, 'range_limit', None)
            } for id, trans in self.transports.items()},
            'cargos': {id: {
                'id': c.id, 'name': c.name, 'weight': c.weight, 'volume': c.volume,
                'fragile': c.fragile, 'hazardous': c.hazardous
            } for id, c in self.cargos.items()},
            'routes': [{
                'from_warehouse_id': r.from_warehouse.id,
                'to_warehouse_id': r.to_warehouse.id,
                'distance': r.distance
            } for r in self.routes]
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

def main():
    manager = TransportationManager()

    warehouse1 = Warehouse("WH001", "Склад Минск", "Минск", 10000)
    warehouse2 = Warehouse("WH002", "Склад Витебск", "Витебск", 8000)
    warehouse3 = Warehouse("WH003", "Склад Брест", "Брест", 12000)
    warehouse4 = Warehouse("WH004", "Склад Гомель", "Гомель", 15000)

    manager.add_warehouse(warehouse1)
    manager.add_warehouse(warehouse2)
    manager.add_warehouse(warehouse3)
    manager.add_warehouse(warehouse4)

    truck1 = Truck("TR001", "Грузовик 1", 50, 20000, 80, 50)
    truck2 = Truck("TR002", "Грузовик 2", 40, 18000, 90, 55)
    ship1 = Ship("SH001", "Корабль", 1000, 500000, 25, 20)
    train1 = Train("TN001", "Поезд", 300, 200000, 60, 30)
    airplane1 = Airplane("AP001", "Самолет", 120, 100000, 900, 1000, 15000)

    manager.add_transport(truck1)
    manager.add_transport(truck2)
    manager.add_transport(ship1)
    manager.add_transport(train1)
    manager.add_transport(airplane1)

    route1 = Route(warehouse1, warehouse2, 700)
    route2 = Route(warehouse1, warehouse3, 1200)
    route3 = Route(warehouse2, warehouse4, 500)
    route4 = Route(warehouse3, warehouse4, 800)

    manager.add_route(route1)
    manager.add_route(route2)
    manager.add_route(route3)
    manager.add_route(route4)

    cargo1 = Cargo("C001", "Электроника", 5000, 50, fragile=True)
    cargo2 = Cargo("C002", "Мебель", 15000, 200, fragile=True)
    cargo3 = Cargo("C003", "Химикаты", 8000, 100, hazardous=True)
    cargo4 = Cargo("C004", "Продукты", 3000, 30, fragile=True)

    manager.add_cargo(cargo1)
    manager.add_cargo(cargo2)
    manager.add_cargo(cargo3)
    manager.add_cargo(cargo4)

    print(" СИСТЕМА УПРАВЛЕНИЯ ПЕРЕВОЗКАМИ")

    print("\n ДОСТУПНЫЕ СКЛАДЫ:")
    for wh in manager.warehouses.values():
        print(f"  • {wh.name} ({wh.location}) - загрузка: {wh.current_load}/{wh.capacity} кг")

    print("\n ДОСТУПНЫЙ ТРАНСПОРТ:")
    for trans in manager.transports.values():
        print(f"  • {trans.name} (грузоподъемность: {trans.max_weight} кг, объем: {trans.capacity} м³)")

    print("\n ДОСТУПНЫЕ МАРШРУТЫ:")
    for route in manager.routes:
        print(f"  • {route.from_warehouse.location} → {route.to_warehouse.location} ({route.distance} км)")

    print("\n ДОСТУПНЫЕ ГРУЗЫ:")
    for cargo in manager.cargos.values():
        print(f"  • {cargo.name} ({cargo.weight} кг, {cargo.volume} м³)")

    print("\n РАСЧЕТ СТОИМОСТИ ПЕРЕВОЗКИ:")
    print("Для груза 'Электроника' по маршруту Минск → Витебск:")
    options = manager.get_delivery_options("C001", "Минск", "Витебск")
    for option in options:
        print(f"  • {option['transport']}: {option['cost']:.2f} руб., время: {option['time']}")

    print("\n РАСЧЕТ ВРЕМЕНИ ДОСТАВКИ:")
    print("Для груза 'Мебель' по маршруту Минск → Брест:")
    options = manager.get_delivery_options("C002", "Минск", "Брест")
    for option in sorted(options, key=lambda x: x['time']):
        print(f"  • {option['transport']}: {option['time']}, стоимость: {option['cost']:.2f} руб.")

    print("\n МУЛЬТИДОСТАВКА:")
    print("Для груза 'Электроника' Минск → Гомель через Витебск:")
    multi_options = manager.get_multi_delivery_options("C001", "Минск", "Гомель")
    for i, option in enumerate(multi_options[:3]):
        print(f"  Вариант {i+1}:")
        print(f"    Маршрут: {' → '.join(option['route_chain'])}")
        print(f"    Транспорт: {' + '.join(option['transports'])}")
        print(f"    Стоимость: {option['cost']:.2f} руб.")
        print(f"    Время: {option['time']}")

    print("\n СОЗДАНИЕ ПРОСТОЙ ДОСТАВКИ:")
    start_time = datetime.now()
    delivery = manager.create_delivery("D001", "C001", "Минск", "Витебск", "TR001", start_time)
    if delivery:
        print(f"  Создана доставка {delivery.id}")
        print(f"  Груз: {delivery.cargo.name}")
        print(f"  Маршрут: {delivery.route.from_warehouse.location} → {delivery.route.to_warehouse.location}")
        print(f"  Транспорт: {delivery.transport.name}")
        print(f"  Стоимость: {delivery.cost:.2f} руб.")
        print(f"  Время доставки: {delivery.delivery_time}")
        print(f"  Статус: {delivery.status}")

    print("\n СОЗДАНИЕ МУЛЬТИДОСТАВКИ:")
    route_chain = [route1, route3]
    transports = ["TR001", "TR002"]
    multi_delivery = manager.create_multi_delivery("MD001", "C001", route_chain, transports, start_time)
    if multi_delivery:
        print(f"  Создана мультидоставка {multi_delivery.id}")
        print(f"  Груз: {multi_delivery.cargo.name}")
        print(f"  Маршрут: {' → '.join([r.from_warehouse.location + ' → ' + r.to_warehouse.location for r in multi_delivery.routes])}")
        print(f"  Транспорт: {' + '.join([t.name for t in multi_delivery.transports])}")
        print(f"  Общая стоимость: {multi_delivery.total_cost:.2f} руб.")
        print(f"  Общее время: {multi_delivery.total_time}")
        print(f"  Статус: {multi_delivery.status}")

    print("\n СОСТОЯНИЕ СКЛАДОВ ПОСЛЕ СОЗДАНИЯ ДОСТАВОК:")
    for wh in manager.warehouses.values():
        print(f"  • {wh.name}: загрузка {wh.current_load}/{wh.capacity} кг")

    manager.save_to_json("transportation_system.json")

if __name__ == "__main__":
    main()

 СИСТЕМА УПРАВЛЕНИЯ ПЕРЕВОЗКАМИ

 ДОСТУПНЫЕ СКЛАДЫ:
  • Склад Минск (Минск) - загрузка: 0.0/10000 кг
  • Склад Витебск (Витебск) - загрузка: 0.0/8000 кг
  • Склад Брест (Брест) - загрузка: 0.0/12000 кг
  • Склад Гомель (Гомель) - загрузка: 0.0/15000 кг

 ДОСТУПНЫЙ ТРАНСПОРТ:
  • Грузовик 1 (грузоподъемность: 20000 кг, объем: 50 м³)
  • Грузовик 2 (грузоподъемность: 18000 кг, объем: 40 м³)
  • Корабль (грузоподъемность: 500000 кг, объем: 1000 м³)
  • Поезд (грузоподъемность: 200000 кг, объем: 300 м³)
  • Самолет (грузоподъемность: 100000 кг, объем: 120 м³)

 ДОСТУПНЫЕ МАРШРУТЫ:
  • Минск → Витебск (700 км)
  • Минск → Брест (1200 км)
  • Витебск → Гомель (500 км)
  • Брест → Гомель (800 км)

 ДОСТУПНЫЕ ГРУЗЫ:
  • Электроника (5000 кг, 50 м³)
  • Мебель (15000 кг, 200 м³)
  • Химикаты (8000 кг, 100 м³)
  • Продукты (3000 кг, 30 м³)

 РАСЧЕТ СТОИМОСТИ ПЕРЕВОЗКИ:
Для груза 'Электроника' по маршруту Минск → Витебск:
  • Корабль: 13300.00 руб., время: 1 day, 4:00:00
  • Поезд: 14700.00 руб.,